In [82]:
import gzip
from collections import defaultdict
import sklearn
from sklearn import linear_model
import string
import re
import datetime, time
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.dates as md
import math
from gensim.models import Word2Vec


In [83]:
def readCSV(path):
    f = open(path, 'rt')
    f.readline()

    for l in f:
        
        if l.startswith("timestamp"): 
            continue

        try: 
            timestamp,company,level,title,totalyearlycompensation,location,yearsofexperience,yearsatcompany,tag,basesalary,stockgrantvalue,bonus,gender,otherdetails,cityid,dmaid,rowNumber,Masters_Degree,Bachelors_Degree,Doctorate_Degree,Highschool,Some_College,Race_Asian,Race_White,Race_Two_Or_More,Race_Black,Race_Hispanic,Race,Education = re.split(r',(?![ ])', l)

        except:
            continue

        yield timestamp,company,level,title,totalyearlycompensation,location,yearsofexperience,yearsatcompany,tag,basesalary,stockgrantvalue,bonus,gender,otherdetails,cityid,dmaid,rowNumber,Masters_Degree,Bachelors_Degree,Doctorate_Degree,Highschool,Some_College,Race_Asian,Race_White,Race_Two_Or_More,Race_Black,Race_Hispanic,Race,Education


In [84]:
salaryRaw = []
for l in readCSV("Levels_Fyi_Salary_Data.csv"):
    salaryRaw.append(l)

In [85]:
print(len(salaryRaw))
#city id = 14, dmaid = 15, city = 5
print(salaryRaw[0][5])

print(salaryRaw[0])



62300
"Redwood City, CA"
('6/7/2017 11:33:27', 'Oracle', 'L3', 'Product Manager', '127000', '"Redwood City, CA"', '1.5', '1.5', 'NA', '107000', '20000', '10000', 'NA', 'NA', '7392', '807', '1', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', 'NA', 'NA\n')


In [86]:
# id , unixtime, company, level, title, total_comp, city, state, experience, tenure, tag, 
# salary, stock, bonus, city_id, dma_id, ms_deg, bs_deg, phd_deg, hs, college

def parseSalaryRaw(salaryRaw):
    salaryAllData = []

    for entry in salaryRaw:

        date, clocktime = entry[0].split(' ')
        date = date.split('/')
        clocktime = clocktime.split(':')
        timestamp = datetime.datetime(int(date[2]), int(date[0]), int(date[1]), int(clocktime[0]), int(clocktime[1]), int(clocktime[2]))
        timestamp = int(time.mktime(timestamp.timetuple()))


        company = entry[1]
        level = entry[2]
        title = entry[3]

        total_comp = int(float(entry[4]))

        location = entry[5]
        if location.count(',') == 2:
            city, state, country = location.strip('"').split(', ')
        elif location.count(',') == 1:
            city, state = location.strip('"').split(', ')
            country = 'USA'

        experience = entry[6]
        tenure = entry[7]

        tag = entry[8]

        salary = int(float(entry[9]))
        if salary == 0:
            continue

        stock = int(float(entry[10]))
        bonus = int(float(entry[11]))

        g = entry[12]
        gender = -1
        if g == 'Female':
            gender = 0
        elif g == 'Male':
            gender = 1
        elif g == 'Other':
            gender = 2

        city_id = int(entry[14])

        try: 
            dma_id = int(entry[15])
        except:
            dma_id = -1

        id = int(entry[16])

        ms_deg = int(entry[17])
        bs_deg = int(entry[18])
        phd_deg = int(entry[18])
        hs = int(entry[19])
        college = int(entry[20])

        r = entry[27]
        race = -1
        if r == 'Asian':
            race = 0
        elif r == 'Black':
            race = 1
        elif r == 'Hispanic':
            race = 2
        elif r == 'Two Or More':
            race = 3
        elif r == 'White':
            race = 4


        salaryAllData.append({
            'id': id,
            'timestamp': timestamp,
            'company': company,
            'level': level,
            'title': title,
            'total_comp': total_comp,
            'city': city,
            'state': state,
            'country': country,
            'experience': experience,
            'tenure': tenure,
            'tag': tag,
            'salary': salary,
            'stock': stock,
            'bonus': bonus,
            'gender': gender,
            'city_id': city_id,
            'dma_id': dma_id,
            'ms_deg': ms_deg,
            'bs_deg': bs_deg,
            'phd_deg': phd_deg,
            'hs': hs,
            'college': college,
            'race': race,
        })
    return salaryAllData

salaryAllData = parseSalaryRaw(salaryRaw)

In [87]:
print(len(salaryAllData))
print(salaryAllData[4]['dma_id'])

59996
819


In [88]:
#####
# Training, Validation, and Test sets
#####

# shuffle data
shuffledAllData = sklearn.utils.shuffle(salaryAllData)
shuffledAllData[0]['id']

# create 80 / 10 / 10 : Train / Validation / Test sets
X = shuffledAllData
y = [d['salary'] for d in shuffledAllData]

X_train, X_temp, y_train, y_temp = sklearn.model_selection.train_test_split(X, y, test_size=0.2, random_state=1)
X_valid, X_test, y_valid, y_test = sklearn.model_selection.train_test_split(X_temp, y_temp, test_size=0.5, random_state=1)

# Validate that sets were created with propper sizes
print(str(len(X_train)) + ", " + str(len(y_train)))
print(str(len(X_valid)) + ", " + str(len(y_valid)))
print(str(len(X_test)) + ", " + str(len(y_test)))

# Validate all ids are different
print(str(X_train[0]['id']) + ", " + str(X_valid[0]['id']) + ", " + str(X_test[0]['id']))

47996, 47996
6000, 6000
6000, 6000
75454, 56026, 38015


In [89]:
X_train[0]

{'id': 75454,
 'timestamp': 1624681235,
 'company': 'eBay',
 'level': 'MTS 1',
 'title': 'Software Engineer',
 'total_comp': 180000,
 'city': 'Toronto',
 'state': 'ON',
 'country': 'Canada',
 'experience': '14',
 'tenure': '0',
 'tag': 'Web Development (Front-End)',
 'salary': 127000,
 'stock': 40000,
 'bonus': 13000,
 'gender': 1,
 'city_id': 1206,
 'dma_id': 0,
 'ms_deg': 0,
 'bs_deg': 1,
 'phd_deg': 1,
 'hs': 0,
 'college': 0,
 'race': 0}

In [90]:
#counting all dma_id just for exploration
#assigning indices for every dma_id
def dma_count_plus_index(dict, ind_dict):
    count = 1
    for d in salaryRaw:
        dict[d[15]] += 1
        if not ind_dict.get(d[15]):
            ind_dict[d[15]] = count
            count += 1
        


In [91]:
dmaidcounts = defaultdict(int)
dmaindices = {}
dma_count_plus_index(dmaidcounts, dmaindices)

In [92]:
print(dmaindices)

{'807': 1, '819': 2, '0': 3, '501': 4, '512': 5, '862': 6, '820': 7, '803': 8, '511': 9, '635': 10, '506': 11, '825': 12, '622': 13, '508': 14, '519': 15, '751': 16, '652': 17, '504': 18, '527': 19, '617': 20, '613': 21, '602': 22, '532': 23, '753': 24, '752': 25, '623': 26, '556': 27, '669': 28, '628': 29, '566': 30, '515': 31, '560': 32, '528': 33, '534': 34, '524': 35, '609': 36, '526': 37, '521': 38, '648': 39, '770': 40, '517': 41, '518': 42, '535': 43, '505': 44, '641': 45, '790': 46, '539': 47, '670': 48, '533': 49, '523': 50, '618': 51, '637': 52, '616': 53, '855': 54, '510': 55, '563': 56, '757': 57, '659': 58, '625': 59, '691': 60, '705': 61, '576': 62, '811': 63, '679': 64, '541': 65, '544': 66, '828': 67, '789': 68, '821': 69, '588': 70, '611': 71, '716': 72, '801': 73, '561': 74, '682': 75, '584': 76, '671': 77, 'NA': 78, '564': 79, '502': 80, '640': 81, '678': 82, '581': 83, '642': 84, '514': 85, '598': 86, '643': 87, '567': 88, '839': 89, '746': 90, '500': 91, '675': 92,

In [93]:
def dma_feat(d, dict):  #returns feature row for dma_id one hot ecoding only
    arr = [1] + ([0]*149)
    if d == '-1':
         arr[77] = 1
    elif d != '807':
        arr[(dict[d]-1)] = 1
    return arr

def dma_yoe_feat(d, dict, exp): #returns row for dma_id and either tenure/yoe
    # print(exp, type(exp))
    arr = [1] + ([0]*149)
    if d == '-1':
         arr[77] = 1
    elif d != '807':
        arr[(dict[d]-1)] = 1
    return arr + [exp]

In [94]:
#constructing feature vectors

X_train_dma = [dma_feat(str(d['dma_id']), dmaindices) for d in X_train]
X_train_dma_yoe = [dma_yoe_feat(str(d['dma_id']), dmaindices, (float(d['experience']))) for d in X_train]
X_train_dma_tenure = [dma_yoe_feat(str(d['dma_id']), dmaindices, (float(d['tenure']))) for d in X_train]


In [95]:
#first model with just dma_id one-hot encoding
model = sklearn.linear_model.LinearRegression(fit_intercept=False)
model.fit(X_train_dma, y_train)
predictions = model.predict(X_train_dma)

In [96]:
#second model with dma_id one-hot encoding + yoe (exeperience)
model2 = sklearn.linear_model.LinearRegression(fit_intercept=False)
model2.fit(X_train_dma_yoe, y_train)
predictions2 = model2.predict(X_train_dma_yoe)

In [97]:
#third model with dma_id one-hot encoding + tenure
model3 = sklearn.linear_model.LinearRegression(fit_intercept=False)
model3.fit(X_train_dma_tenure, y_train)
predictions3 = model3.predict(X_train_dma_tenure)

In [98]:
print(predictions[:10])
print(predictions2[:10])
print(predictions3[:10])
y_train[:10]

[ 80151.02983512 172283.71819008 151613.63085812 151613.63085812
 156421.88358731 151613.63085812 133338.95131086  80151.02983512
 151613.63085812 172283.71819008]
[109046.57502351 171114.04233288 120736.57824752 120736.57824752
 142518.40224884 153701.82360145 113737.86426495  67840.01833111
 170184.44627841 142269.4526482 ]
[ 72220.42659865 164000.13755512 141934.92387229 141934.92387229
 158580.14319045 141934.92387229 132839.36830457  75504.85061648
 148503.77190795 164000.13755512]


[127000, 200000, 125000, 80000, 166000, 152000, 79000, 55000, 150000, 120000]

In [99]:
# for key,value in dmaindices.items():
    
#     print(key, value)

In [100]:
# print(len(dmaidcounts))

# dmaidcounts['807']

In [101]:
# for key,value in dmaidcounts.items():
#     print(key, value)

In [102]:
def MSE(predictions, labels):
    differences = [(x-y)**2 for x,y in zip(predictions,labels)]
    return sum(differences) / len(differences)

In [103]:
def percent_err(y, pred):
    tot = 0
    for actual, pred in zip(y, pred):
        tot += abs(100 * ((pred-actual) / pred))
    avg = tot/len(y)
    return avg

In [104]:
#create feature vector for X_valid

X_valid_dma = [dma_feat(str(d['dma_id']), dmaindices) for d in X_valid]
X_valid_dma_yoe = [dma_yoe_feat(str(d['dma_id']), dmaindices, (float(d['experience']))) for d in X_valid]
X_valid_dma_tenure = [dma_yoe_feat(str(d['dma_id']), dmaindices, (float(d['tenure']))) for d in X_valid]


In [105]:
valid_predictions_dma = model.predict(X_valid_dma)
valid_predictions_dma_yoe = model2.predict(X_valid_dma_yoe)
valid_predictions_dma_tenure = model3.predict(X_valid_dma_tenure)


In [106]:
print(valid_predictions_dma[:10])

print(valid_predictions_dma_yoe[:10])

print(valid_predictions_dma_tenure[:10])

print(y_valid[:10])

[151613.63085812 150484.98062016 172283.71819008 172283.71819008
 172283.71819008 118914.46345256 150484.98062016 172283.71819008
 110626.5060241  131155.15463917]
[178425.75761689 183123.60811053 171114.04233288 150510.76398668
 162872.7309944  171515.30595901 129555.0844104  150510.76398668
 123584.23045931 106335.63839589]
[174779.1640506  142215.67937085 177137.83362644 167284.56157295
 167284.56157295 113224.18613301 148784.52740651 170568.98559078
 120638.06409587 124975.69737072]
[160000, 154000, 178000, 125000, 170000, 155000, 111000, 160000, 215000, 92000]


In [107]:
percent_err(y_valid, valid_predictions_dma )

24.053771384386184

In [108]:
MSE(y_valid, valid_predictions_dma )

1846770210.7759118